# UNICEF DATAVIZ CHALLENGE - DATA PREPARATION

Loading datasets

In [2]:
import pandas as pd


summary = pd.read_excel("data/summary_survey_ covid_response.xlsx", sheet_name="Long")
detailed = pd.read_excel("data/detail_survey_covid_response.xlsx", sheet_name="Data")
atttendance_pre_primary = pd.read_excel("data/attendance_nov2019.xlsx", sheet_name="ANAR one year before")
atttendance_primary = pd.read_excel("data/attendance_nov2019.xlsx", sheet_name="Primary ANAR")
atttendance_lower_secondary = pd.read_excel("data/attendance_nov2019.xlsx", sheet_name="Lower Secondary ANAR")
atttendance_upper_secondary = pd.read_excel("data/attendance_nov2019.xlsx", sheet_name="Upper Secondary ANAR")
completion_primary   = pd.read_excel("data/completion_nov2019.xlsx", sheet_name="primary")
completion_lower_secondary   = pd.read_excel("data/completion_nov2019.xlsx", sheet_name="lower secondary")
completion_upper_secondary  = pd.read_excel("data/completion_nov2019.xlsx", sheet_name="upper secondary")

Data Cleaning on summary responses from governement covid-19 responses survey

In [ ]:
# only keep education subject 
summary = summary[summary["pageTitle"] == "Educational socio-economic impacts and national response"]

#only keep the question and answers when  asking for the sucess level in implementing : remote learning, paper based booklets, teacher support, monitoring student learning system

summary = summary[(summary["parentQuestion"] == "Please indicate the level of success the country experienced in the following aspects of implementing remote learning (such as radio, TV, online platforms, paper-based exercises etc.):") & ((summary["question"] == "Access to remote learning") | (summary["question"] ==  "Tracking or monitoring of student learning") | (summary["question"] ==  "Coverage of curriculum (subjects and content planned)") | (summary["question"] ==  "Teacher support (pedagogical, psychosocial, financial, etc.)"))]

In [ ]:
# keep only useful  columns for the vizualization 

summary = summary[['category', 'subCategory', 'ISO Code', 'question', 'value']]

In [ ]:
# get one single record by country regarding each level of sucess studied for the analysis
manip  = summary[['ISO Code', 'question', 'value']]
manip =  manip.pivot(index='ISO Code', columns='question').add_prefix('value_')
manip.columns = manip.columns.droplevel(0)
manip = manip.reset_index()
summary_cleaned = pd.merge(manip, summary[['category', 'subCategory', 'ISO Code']], on='ISO Code', how='inner') # re fetching category and subcategory with inner join
summary_cleaned = summary_cleaned.drop_duplicates() #removing duplicated row created by the merge



In [ ]:
#pushing the dataframe object into a csv file for tableau analysis
summary_cleaned.to_csv("summary_Covid19_educational_survey_cleaned.csv", sep=";")


Data Cleaning on detailed responses from governement covid-19 responses survey

In [ ]:
# questions selected for the analysis were manually selecting and are the following : 

In [ ]:
# working on Q13 : distance learning plateforms effectiveness 
Q13  = detailed[['ISO3','Q13Onlineplatforms_Effectiven', 'Q13Television_Effectiveness', 'Q13Radio_Effectiveness', 'Q13Take_homepackages_Effectiv']]
Q13 = Q13.fillna('null')
Q13['Media'] = "null"

for i in range(0,len(Q13)):
    Q13.at[i,'Media'] = ['Online learning','Television','Radio','Take Home Packages']

Q13['Q13Answers'] = Q13['Q13Onlineplatforms_Effectiven']+","+Q13['Q13Television_Effectiveness']+","+Q13['Q13Radio_Effectiveness']+","+Q13['Q13Take_homepackages_Effectiv']

Q13['Effectiveness'] = "null"
for i in range(0,len(Q13)):
    Q13.at[i,'Effectiveness'] = Q13['Q13Answers'][i].split(",")

Q13 = Q13[['ISO3', 'Media','Effectiveness']]
#exploding one single record into multiple records for a country : transform Q13Categories and Q13Answers into categorical variables 
import numpy as np
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res


Q13 =  explode(Q13, ['Media','  Effectiveness'], fill_value='')
Q13.to_csv("Distance_Learning_media_effectiveness.csv", sep =";")  # in excel we also replace "We do not have such plateform by Media not available for vizualization purpose"



In [ ]:
# working on Q16: measures took to improve access to distance  learning 
Q16 = detailed[['ISO3','Q16Offernegotiateaccesstothe', 'Q16landline', 'Q16mobilephones', 'Q16Subsidizedfreedevicesfora','Q16Nomeasurestaken', 'Q16Donotknow']]
Q16 = Q16.fillna('null')
Q16['Measures'] = "null"

for i in range(0,len(Q16)):
    Q16.at[i,'Measures'] = ['Offer/negotiate access to the internet','Access via Landlines','Access via Mobile Phones','Subsidized/free devices for access', 'No measures taken', 'Do not know']

Q16['Q16Answers'] = Q16['Q16Offernegotiateaccesstothe']+","+Q16['Q16landline']+","+Q16['Q16mobilephones']+","+Q16['Q16Subsidizedfreedevicesfora']+","+Q16['Q16Nomeasurestaken']+","+Q16['Q16Donotknow']

Q16['Answers'] = "null"
for i in range(0,len(Q16)):
    Q16.at[i,'Answers'] = Q16['Q16Answers'][i].split(",")

Q16 = Q16[['ISO3', 'Measures','Answers']]
#exploding one single record into multiple records for a country regarding its answers to Q16
Q16 =  explode(Q16, ['Measures','Answers'], fill_value='')
Q16.to_csv("Measures_access_distance_learning.csv", sep =";")  

In [26]:
#working on Q17: insigths by country on online platform used for remote learning 
import re

Q17 = detailed[['ISO3','Q17_2PlatformcreatedbytheMin','Q17_2Commercialplatformnotfor','Q17_2CommercialforfreeBlackb','Q17_2OpensourceplatformMoodl','Q17_2Donotknow']]
Q17 = Q17.fillna('null')

Q17["Q17_2PlatformcreatedbytheMin"] = Q17["Q17_2PlatformcreatedbytheMin"].replace(["Platform created by the Ministry of Education or education authorities (If yes, answer Question 17.2.1-17.2.3)"], "Platform created by education authorities")
Q17["Q17_2Commercialplatformnotfor"] = Q17["Q17_2Commercialplatformnotfor"].replace(["Commercial platform not for free (Microsoft Teams, etc.) (If yes, answer Question 17.2.4)"], "Paying Commercial platforms (Microsoft Teams, ect.)")


Q17['PlatformsUsed'] = Q17['Q17_2PlatformcreatedbytheMin']+";"+Q17['Q17_2Commercialplatformnotfor']+";"+Q17['Q17_2CommercialforfreeBlackb']+";"+Q17['Q17_2OpensourceplatformMoodl']+";"+Q17['Q17_2Donotknow']

Q17['PlatformsUsed'] =Q17['PlatformsUsed'].apply(lambda x: re.sub("null", "", string=x))
Q17['PlatformsUsed'] =Q17['PlatformsUsed'].apply(lambda x: re.sub(";;;;", "", string=x))
Q17['PlatformsUsed'] =Q17['PlatformsUsed'].apply(lambda x: re.sub("^;;", "", string=x))
Q17['PlatformsUsed'] =Q17['PlatformsUsed'].apply(lambda x: re.sub("^;", "", string=x))
Q17['PlatformsUsed'] =Q17['PlatformsUsed'].apply(lambda x: re.sub(";;$", "", string=x))
Q17['PlatformsUsed'] =Q17['PlatformsUsed'].apply(lambda x: re.sub(";$", "", string=x))
Q17['PlatformsUsed'] =Q17['PlatformsUsed'].apply(lambda x: re.sub(";;;", ";", string=x))
Q17['PlatformsUsed'] =Q17['PlatformsUsed'].apply(lambda x: re.sub(";;", ";", string=x))
Q17['PlatformsUsed'] =Q17['PlatformsUsed'].apply(lambda x: re.sub(";", "; ", string=x))

Q17 = Q17[["ISO3", "PlatformsUsed"]]
Q17["PlatformsUsed"] = Q17["PlatformsUsed"].replace(['', "Do not know"], "NO INFORMATION")
Q17.to_csv("Online_Platforms_used.csv", sep = ";", index= False)


In [ ]:
# working ON Q15 : If remote learning is considered as sufficient as traditional learning 
#simply extracting the columns and the ISO code associated : no cleaning needed 
Q15 = detailed[['ISO3', 'Q15']]
Q15 = Q15.rename(columns = {'Q15': "Remote_Learning_vs_Traditional Learning"})
Q15 = Q15.fillna('Do not know')
Q15.to_csv('Remote_Learning_vs_Traditional_Learning.csv', sep = ";", index=False)


In [ ]:
#working on Q12 and Q2 : estimate number of days of schools skipped by class levels (whether school year was over or not)
Q2_Q12 = detailed[['ISO3', 'Q2Primaryeducation', 'Q2Lower_secondaryeducation', 'Q2Upper_secondaryeducation', 'Q12_1primaryeducation', 'Q12_1lower_secondaryeducation', 'Q12_1upper_secondaryeducation']]
Q2_Q12 = Q2_Q12.fillna("null") # filling NA value with null (countries who did not answers )
#replacing the 9999, 999, 0 value by null (answers when countries did not know nb of days for a school year) and converting the columns into string type 
for c in ['Q2Primaryeducation', 'Q2Lower_secondaryeducation', 'Q2Upper_secondaryeducation']:
    Q2_Q12[c] = Q2_Q12[c].replace([9999, 999, 0], "null")
    Q2_Q12[c] = Q2_Q12[c].astype('str')

for c in ['Q12_1primaryeducation', 'Q12_1lower_secondaryeducation', 'Q12_1upper_secondaryeducation']:
    Q2_Q12[c] = Q2_Q12[c].astype('str')


Q2_Q12['EducationLevel'] = "null"

for i in range(0,len(Q2_Q12)):
    Q2_Q12.at[i,'EducationLevel'] = ['Primary Education', 'Lower Secondary Education', 'Upper Secondary Education']

Q2_Q12['Q2Answers'] = Q2_Q12['Q2Primaryeducation']+","+Q2_Q12['Q2Lower_secondaryeducation']+","+Q2_Q12['Q2Upper_secondaryeducation']

Q2_Q12['Q12Answers'] = Q2_Q12['Q12_1primaryeducation']+","+Q2_Q12['Q12_1lower_secondaryeducation']+","+Q2_Q12['Q12_1upper_secondaryeducation']

print(Q2_Q12['Q2Answers'])
Q2_Q12['SchoolYearDays'] = "null"
Q2_Q12['SchoolDaysSkipped'] = "null"

for i in range(0,len(Q2_Q12)):
    Q2_Q12.at[i,'SchoolYearDays'] = Q2_Q12['Q2Answers'][i].split(",")
    Q2_Q12.at[i,'SchoolDaysSkipped'] = Q2_Q12['Q12Answers'][i].split(",")

Q2_Q12 = Q2_Q12[['ISO3', 'EducationLevel','SchoolYearDays', 'SchoolDaysSkipped']]
#exploding one single record into multiple records for a country regarding its answers to Q2 and Q12 
Q2_Q12 =  explode(Q2_Q12, ['EducationLevel','SchoolYearDays', 'SchoolDaysSkipped'], fill_value='')
#reconverting Nb of School days in a year and Nb of days skippes  into float columns 
#for c in  ['SchoolYearDays', 'SchoolDaysSkipped']:
    #Q2_Q12[c] = Q2_Q12[c].astype('float')
    
Q2_Q12.to_csv("Schools_days_skipped_v2.csv", sep =";", index=False)  



In [ ]:
Q2_Q12.dtypes

In [42]:
# working on Q20 : measures to support teachers during the crisis

Q20 = detailed[['ISO3','Q20OfferedspecialtrainingIf', 'Q20Providedwithinstructionon', 'Q20Providedwithprofessionalp', 'Q20Providedwithteachingconten','Q20ProvidedwithICTtoolsandf', 'Q20Noadditionalsupportwasoff', 'Q20Donotknow']]
Q20 = Q20.fillna('null')

for c in ['Q20OfferedspecialtrainingIf', 'Q20Providedwithinstructionon', 'Q20Providedwithprofessionalp', 'Q20Providedwithteachingconten','Q20ProvidedwithICTtoolsandf', 'Q20Noadditionalsupportwasoff', 'Q20Donotknow']:
    Q20[c] = Q20[c].astype('str')



Q20["Q20OfferedspecialtrainingIf"] = Q20["Q20OfferedspecialtrainingIf"].replace(["Offered special training (If so, please answer Question 20.1)"], "Special training offers")

Q20["Q20Providedwithinstructionon"] = Q20["Q20Providedwithinstructionon"].replace(["Provided with instruction on distance instruction (TV, radio, learning platforms, etc.)"], "Distance learning instructions")

Q20["Q20Providedwithprofessionalp"] = Q20["Q20Providedwithprofessionalp"].replace(["Provided with professional, psychosocial and emotional support (e.g. chat groups, online forums to share ideas and educational content)"], "Psychosocial/emotional support")

Q20["Q20Providedwithteachingconten"] = Q20["Q20Providedwithteachingconten"].replace(["Provided with teaching content adapted to remote teaching (e.g. use of open educational resources (OERs), sample lesson plans etc..)"], "Adapted contents for remote teaching")

Q20["Q20ProvidedwithICTtoolsandf"] = Q20["Q20ProvidedwithICTtoolsandf"].replace(["Provided with ICT tools and free connectivity (PC, mobile device, voucher for mobile broadband, etc..)"], "Free ICT tools and connectivity")

Q20["Q20Noadditionalsupportwasoff"] = Q20["Q20Noadditionalsupportwasoff"].replace(["No additional support was offered to teachers"], "No support received")


Q20['SupportToTeachers'] = Q20['Q20OfferedspecialtrainingIf']+";"+Q20['Q20Providedwithinstructionon']+";"+Q20['Q20Providedwithprofessionalp']+";"+Q20['Q20Providedwithteachingconten']+";"+Q20['Q20ProvidedwithICTtoolsandf']+";"+Q20['Q20Noadditionalsupportwasoff']+";"+Q20['Q20Donotknow']


Q20['SupportToTeachers'] =Q20['SupportToTeachers'].apply(lambda x: re.sub("null", "", string=x))
Q20['SupportToTeachers'] =Q20['SupportToTeachers'].apply(lambda x: re.sub(";;;;", "", string=x))
Q20['SupportToTeachers'] =Q20['SupportToTeachers'].apply(lambda x: re.sub("^;;", "", string=x))
Q20['SupportToTeachers'] =Q20['SupportToTeachers'].apply(lambda x: re.sub("^;", "", string=x))
Q20['SupportToTeachers'] =Q20['SupportToTeachers'].apply(lambda x: re.sub(";;$", "", string=x))
Q20['SupportToTeachers'] =Q20['SupportToTeachers'].apply(lambda x: re.sub(";$", "", string=x))
Q20['SupportToTeachers'] =Q20['SupportToTeachers'].apply(lambda x: re.sub(";;;", ";", string=x))
Q20['SupportToTeachers'] =Q20['SupportToTeachers'].apply(lambda x: re.sub(";;", ";", string=x))
Q20['SupportToTeachers'] =Q20['SupportToTeachers'].apply(lambda x: re.sub(";", "; ", string=x))

Q20["SupportToTeachers"] = Q20["SupportToTeachers"].replace(["", "Do not know"], "NO INFORMATION")

Q20['SupportToTeachers'] =Q20['SupportToTeachers'].apply(lambda x: re.sub("Do not know", "", string=x))
Q20['SupportToTeachers'] =Q20['SupportToTeachers'].apply(lambda x: re.sub("No support received", "", string=x))

Q20["SupportToTeachers"] = Q20["SupportToTeachers"].replace([""], "No support received")
Q20 = Q20[['ISO3','SupportToTeachers']]

Q20['SupportToTeachers'].value_counts()
Q20.to_csv("Measures_support_to_teachers.csv", sep =";", index=False)  


In [ ]:
# working on Q21 : measures to support teachers during the crisis

Q21 = detailed[['ISO3','Q21Phonecallstostudentsorpa', 'Q21Emailstostudents', 'Q21TextWhatsAppotherapplicati', 'Q21Homevisits','Q21Therewerenospecificguidel', 'Q21Donotknow']]
Q21 = Q21.fillna('null')
Q21['MeasuresToStudents'] = "null"

for i in range(0,len(Q21)):
    Q21.at[i,'MeasuresToStudents'] = ['Phone Calls','Emails','Messaging Applications','Home Visits', 'No measures taken', 'Do not know']

Q21['Q21Answers'] = Q21['Q21Phonecallstostudentsorpa']+","+Q21['Q21Emailstostudents']+","+Q21['Q21TextWhatsAppotherapplicati']+","+Q21['Q21Homevisits']+","+Q21['Q21Therewerenospecificguidel']+","+Q21['Q21Donotknow']

Q21['Contact_students'] = "null"
for i in range(0,len(Q21)):
    Q21.at[i,'Contact_students'] = Q21['Q21Answers'][i].split(",")

Q21 = Q21[['ISO3', 'MeasuresToStudents','Contact_students']]
#exploding one single record into multiple records for a country regarding its answers to Q21
Q21 =  explode(Q21, ['MeasuresToStudents','Contact_students'], fill_value='')
Q21.to_csv("Measures_improve_contact_with_students.csv", sep =";")  


In [ ]:
#working on Q18 : were teachers required to keep teaching during school closures ?
#simply extracting the columns and the ISO code associated : no cleaning needed 
Q18 = detailed[['ISO3', 'Q18']]
Q18 = Q18.rename(columns = {'Q18': "RequiredToTeachDuringClosures"})
Q18 = Q18.fillna('Do not know')
Q18["RequiredToTeachDuringClosures"] = Q18["RequiredToTeachDuringClosures"].replace(["No"], "No requirements")
Q18.to_csv('Teacher_requirement_keep_Teaching_school_closures.csv', sep = ";", index=False)



In [61]:
#working on Q25 : measure to support inclusion in distance learning 
Q25 = detailed[['ISO3','Q25Supporttolearnerswithdisa', 'Q25Improvedaccesstoinfrastruc', 'Q25Designoflearningmaterials', 'Q25Subsidizeddevicesforaccess','Q25Flexibleandself_pacedplatf', 'Q25Specialeffortstomakeonlin', 'Q25Additionalsupporttolower_i', 'Q25None', 'Q25Donotknow']]

Q25 = Q25.fillna('null')

for c in ['Q25Supporttolearnerswithdisa', 'Q25Improvedaccesstoinfrastruc', 'Q25Designoflearningmaterials', 'Q25Subsidizeddevicesforaccess','Q25Flexibleandself_pacedplatf', 'Q25Specialeffortstomakeonlin', 'Q25Additionalsupporttolower_i', 'Q25None', 'Q25Donotknow']:
    Q25[c] = Q25[c].astype('str')


Q25["Q25Supporttolearnerswithdisa"] = Q25["Q25Supporttolearnerswithdisa"].replace(["Support to learners with disabilities (e.g. sign language in online learning programmes)"], "Support to learners with disabilities")

Q25["Q25Improvedaccesstoinfrastruc"] = Q25["Q25Improvedaccesstoinfrastruc"].replace(["Improved access to infrastructure for learners in remote areas; and in urban high-density areas"], "Infrastructure access improved for learner in remote or urban high-density area ")

Q25["Q25Flexibleandself_pacedplatf"] = Q25["Q25Flexibleandself_pacedplatf"].replace(["Flexible and self-paced platforms (Asynchronous learning platforms)"], "Asynchronus learning platforms")

Q25["Q25Specialeffortstomakeonlin"] = Q25["Q25Specialeffortstomakeonlin"].replace(["Special efforts to make online learning more accessible to migrant and displaced children, including those in camps"], "Make online learning accessible to migrand and displaced children")

Q25["Q25Additionalsupporttolower_i"] = Q25["Q25Additionalsupporttolower_i"].replace(["Additional support to lower-income households, including economic support (i.e. take-home rations, cash based transfers)"], "Economic support")

Q25["Q25None"] = Q25["Q25None"].replace(["None"], "No measures to insure inclusion")


Q25['SupportInclusion'] = Q25['Q25Supporttolearnerswithdisa']+";"+Q25['Q25Improvedaccesstoinfrastruc']+";"+Q25['Q25Designoflearningmaterials']+";"+Q25['Q25Subsidizeddevicesforaccess']+";"+Q25['Q25Flexibleandself_pacedplatf']+";"+Q25['Q25Specialeffortstomakeonlin']+";"+Q25['Q25Additionalsupporttolower_i']+";"+Q25['Q25None']+";"+Q25['Q25Donotknow']


Q25['SupportInclusion'] =Q25['SupportInclusion'].apply(lambda x: re.sub("null", "", string=x))
Q25['SupportInclusion'] =Q25['SupportInclusion'].apply(lambda x: re.sub(";;;;", "", string=x))
Q25['SupportInclusion'] =Q25['SupportInclusion'].apply(lambda x: re.sub("^;;", "", string=x))
Q25['SupportInclusion'] =Q25['SupportInclusion'].apply(lambda x: re.sub("^;", "", string=x))
Q25['SupportInclusion'] =Q25['SupportInclusion'].apply(lambda x: re.sub(";;$", "", string=x))
Q25['SupportInclusion'] =Q25['SupportInclusion'].apply(lambda x: re.sub(";$", "", string=x))
Q25['SupportInclusion'] =Q25['SupportInclusion'].apply(lambda x: re.sub(";;;", ";", string=x))
Q25['SupportInclusion'] =Q25['SupportInclusion'].apply(lambda x: re.sub(";;", ";", string=x))
Q25['SupportInclusion'] =Q25['SupportInclusion'].apply(lambda x: re.sub(";", "; ", string=x))

Q25["SupportInclusion"] = Q25["SupportInclusion"].replace(["", "Do not know"], "NO INFORMATION")
Q25['SupportInclusion'] =Q25['SupportInclusion'].apply(lambda x: re.sub("Do not know", "", string=x))
Q25['SupportInclusion'] =Q25['SupportInclusion'].apply(lambda x: re.sub("No measures to insure inclusion", "", string=x))

Q25["SupportInclusion"] = Q25["SupportInclusion"].replace([""], "No measures to insure inclusion")
Q25["SupportInclusion"] = Q25["SupportInclusion"].replace(["; "], "NO INFORMATION")

Q25 = Q25 [['ISO3', 'SupportInclusion']]

Q25.to_csv("Support_inclusion_distance_learning.csv", sep =";", index=False)  

In [65]:
#working on Q26 : 


Q26 = detailed[['ISO3','Q26_1Psychosocialandmentalhea', 'Q26_1Additionalchildprotection', 'Q26_1Supporttocounterinterrup', 'Q26_1Regularcallsfromteachers','Q26_1Nomeasures', 'Q26_1Donotknow']]
Q26 = Q26.fillna('null')
Q26['MeasuresWellBeing_Categories'] = "null"

for i in range(0,len(Q26)):
    Q26.at[i,'MeasuresWellBeing_Categories'] = ['Psycho/Mental support','Child Protection Service','Support for interrupted meal services','Regular Calls from pricipals', 'No measures', 'Do not know']

Q26['Q26Answers'] = Q26['Q26_1Psychosocialandmentalhea']+";"+Q26['Q26_1Additionalchildprotection']+";"+Q26['Q26_1Supporttocounterinterrup']+";"+Q26['Q26_1Regularcallsfromteachers']+";"+Q26['Q26_1Nomeasures']+";"+Q26['Q26_1Donotknow']

Q26['MeasuresWellBeingTaken'] = "null"
for i in range(0,len(Q26)):
    Q26.at[i,'MeasuresWellBeingTaken'] = Q26['Q26Answers'][i].split(";")

Q26 = Q26[['ISO3', 'MeasuresWellBeing_Categories','MeasuresWellBeingTaken']]
#exploding one single record into multiple records for a country regarding its answers to Q26
Q26 =  explode(Q26, ['MeasuresWellBeing_Categories','MeasuresWellBeingTaken'], fill_value='')
Q26.to_csv("Measures_well_being_students.csv", sep =";", index= False)  


In [66]:
#working on Q28
Q28 = detailed[['ISO3','Q28learningmanagementbythesc', 'Q28learningmanagementbythepr', 'Q28TrackingstudentonExcelor', 'Q28Trackingstudentonpaper','Q28Donotknow', 'Q28Progressisnotbeingtracked']]
Q28 = Q28.fillna('null')
Q28['MeasuresTracking_Categories'] = "null"

for i in range(0,len(Q28)):
    Q28.at[i,'MeasuresTracking_Categories'] = ['Tracking system from school ','Commercial Tracking system ','Tracking system on Excel','Paper-based tracking system','Do not know', 'No tracking system']

Q28['Q28Answers'] = Q28['Q28learningmanagementbythesc']+";"+Q28['Q28learningmanagementbythepr']+";"+Q28['Q28TrackingstudentonExcelor']+";"+Q28['Q28Trackingstudentonpaper']+";"+Q28['Q28Donotknow']+";"+Q28['Q28Progressisnotbeingtracked']

Q28['MeasuresTrackingTaken'] = "null"
for i in range(0,len(Q28)):
    Q28.at[i,'MeasuresTrackingTaken'] = Q28['Q28Answers'][i].split(";")

Q28 = Q28[['ISO3', 'MeasuresTracking_Categories','MeasuresTrackingTaken']]
#exploding one single record into multiple records for a country regarding its answers to Q28
Q28 =  explode(Q28, ['MeasuresTracking_Categories','MeasuresTrackingTaken'], fill_value='')
Q28.to_csv("Measures_tracking_students.csv", sep =";", index= False)  


CLeaning november 2019 educational indicators from UNICEF (Attendance and completion rates) and merging them 

In [147]:
atttendance_pre_primary = atttendance_pre_primary[['ISO_Code','Total Population, one year before primary']]
atttendance_pre_primary = atttendance_pre_primary.rename(columns = {'ISO_Code': "ISO"})
atttendance_lower_secondary = atttendance_lower_secondary[['ISO_Code','Total Population, lsec']]
atttendance_upper_secondary = atttendance_upper_secondary[['ISO_Code','Total Population, upper secondary']]

attendance = pd.concat([atttendance_pre_primary,atttendance_primary,atttendance_lower_secondary, atttendance_upper_secondary], axis=1, join='outer')
attendance = attendance[['ISO','Total Population, one year before primary', 'Total Population, primary', 'Total Population, lsec', 'Total Population, upper secondary']]
attendance =attendance.fillna(0)

attendance["Total Attendance Nov2019"] = attendance['Total Population, one year before primary']+attendance['Total Population, primary']+attendance['Total Population, lsec']+attendance['Total Population, upper secondary']


In [148]:
attendance["Total Attendance Nov2019"] = attendance["Total Attendance Nov2019"].replace([0], "null")

In [149]:
attendance

,ISO,"Total Population, one year before primary","Total Population, primary","Total Population, lsec","Total Population, upper secondary",Total Attendance Nov2019
0,AFG,1049189.0,6170913.0,2839981.0,2597554,1.26576e+07
1,ALB,0.0,156833.0,143167.0,131436,431436
2,DZA,887565.0,4040275.0,2479484.0,1692909,9.10023e+06
3,AND,0.0,0.0,0.0,0,null
4,AGO,991313.0,5482624.0,2321369.0,2043253,1.08386e+07
...,...,...,...,...,...,...
197,VEN,593601.0,3530439.0,1727860.0,1134003,6.9859e+06
198,VNM,1527560.0,7363689.0,5380477.0,3935885,1.82076e+07
199,YEM,770587.0,4383983.0,1993572.0,1881485,9.02963e+06
200,ZMB,516416.0,3363211.0,865269.0,1200386,5.94528e+06


In [150]:
import numpy  as np
completion_primary = completion_primary[['ISO_Code','Total']]
completion_primary = completion_primary.rename(columns = {'ISO_Code': "ISO_completion", "Total": "%Total_primary"})
completion_lower_secondary = completion_lower_secondary[['ISO_Code','Total']]
completion_lower_secondary = completion_lower_secondary.rename(columns = {"Total": "%Total_lower_secondary"})
completion_upper_secondary = completion_upper_secondary[['ISO_Code','Total']]
completion_upper_secondary = completion_upper_secondary.rename(columns = {"Total": "%Total_upper_secondary"})

completion = pd.concat([completion_primary,completion_lower_secondary, completion_upper_secondary], axis=1, join='outer')
completion= completion[['ISO_completion', '%Total_primary', '%Total_lower_secondary', '%Total_upper_secondary']]
completion =completion.fillna(0)
completion["%Total_primary"] = completion["%Total_primary"].replace([0], np.nan)
completion["%Total_lower_secondary"] = completion["%Total_lower_secondary"].replace([0], np.nan)
completion["%Total_upper_secondary"] = completion["%Total_upper_secondary"].replace([0], np.nan)
completion["Average % Attendance Nov2019"]  = completion.mean(numeric_only=True, axis=1)



In [151]:
completion = completion.drop(202)
completion = completion.drop(203)

In [152]:
indicators_nov2019 = pd.concat([attendance, completion], axis=1, join='outer')
indicators_nov2019 = indicators_nov2019.drop(columns=['ISO_completion'])


,ISO,"Total Population, one year before primary","Total Population, primary","Total Population, lsec","Total Population, upper secondary",Total Attendance Nov2019,%Total_primary,%Total_lower_secondary,%Total_upper_secondary,Average % Attendance Nov2019
0,AFG,1049189.0,6170913.0,2839981.0,2597554,1.26576e+07,53.900000,37.217084,22.895344,38.004142
1,ALB,0.0,156833.0,143167.0,131436,431436,91.942408,96.245591,50.459568,79.549189
2,DZA,887565.0,4040275.0,2479484.0,1692909,9.10023e+06,93.800000,64.000000,37.800000,65.200000
3,AND,0.0,0.0,0.0,0,null,NaN,NaN,NaN,NaN
4,AGO,991313.0,5482624.0,2321369.0,2043253,1.08386e+07,50.900000,35.685495,17.412797,34.666097
...,...,...,...,...,...,...,...,...,...,...
197,VEN,593601.0,3530439.0,1727860.0,1134003,6.9859e+06,NaN,NaN,NaN,NaN
198,VNM,1527560.0,7363689.0,5380477.0,3935885,1.82076e+07,96.600000,83.700000,55.500000,78.600000
199,YEM,770587.0,4383983.0,1993572.0,1881485,9.02963e+06,62.600000,46.300000,29.500000,46.133333
200,ZMB,516416.0,3363211.0,865269.0,1200386,5.94528e+06,74.100000,50.500000,27.600000,50.733333


In [153]:
indicators_nov2019.to_csv("educational_indicators_nov2019.csv", sep=";", index = False)

Cleaning on polygone data for the globe representation : making sure countries have the same labels as in the survey data 

In [20]:
summary
lands  = pd.read_csv("data/polygones_countries.csv")

In [21]:
unicef = list(summary['subCategory'].unique())
land = list(lands['ADMIN'].unique())
land



['Indonesia',
 'Malaysia',
 'Chile',
 'Bolivia',
 'Peru',
 'Argentina',
 'Dhekelia Sovereign Base Area',
 'Cyprus',
 'India',
 'China',
 'Israel',
 'Palestine',
 'Lebanon',
 'Ethiopia',
 'South Sudan',
 'Somalia',
 'Kenya',
 'Pakistan',
 'Malawi',
 'United Republic of Tanzania',
 'Syria',
 'Somaliland',
 'France',
 'Suriname',
 'Guyana',
 'South Korea',
 'North Korea',
 'Morocco',
 'Western Sahara',
 'Costa Rica',
 'Nicaragua',
 'Republic of the Congo',
 'Democratic Republic of the Congo',
 'Bhutan',
 'Ukraine',
 'Belarus',
 'Namibia',
 'South Africa',
 'Saint Martin',
 'Sint Maarten',
 'Oman',
 'Uzbekistan',
 'Kazakhstan',
 'Tajikistan',
 'Lithuania',
 'Brazil',
 'Uruguay',
 'Mongolia',
 'Russia',
 'Czechia',
 'Germany',
 'Estonia',
 'Latvia',
 'Norway',
 'Sweden',
 'Finland',
 'Vietnam',
 'Cambodia',
 'Luxembourg',
 'United Arab Emirates',
 'Belgium',
 'Georgia',
 'Macedonia',
 'Albania',
 'Azerbaijan',
 'Kosovo',
 'Turkey',
 'Spain',
 'Laos',
 'Kyrgyzstan',
 'Armenia',
 'Denmark',
 

In [22]:
for c in unicef:
    if c in land: print(c, "OK")
    else: print(c, "NOT IN LANDS")

Afghanistan OK
Albania OK
Anguilla OK
Antigua and Barbuda OK
Armenia OK
Bangladesh OK
Barbados OK
Belarus OK
Belize OK
Benin OK
Bhutan OK
Bolivia (Plurinational State of) NOT IN LANDS
Bosnia and Herzegovina OK
Botswana OK
Brazil OK
British Virgin Islands OK
Bulgaria OK
Burkina Faso OK
Burundi OK
Cameroon OK
Central African Republic OK
Chad OK
Chile OK
Colombia OK
Comoros OK
Congo NOT IN LANDS
Cook Islands OK
Costa Rica OK
Croatia OK
Cuba OK
Democratic Republic of the Congo OK
Djibouti OK
Dominica OK
Dominican Republic OK
Ecuador OK
Egypt OK
El Salvador OK
Equatorial Guinea OK
Eswatini NOT IN LANDS
Ethiopia OK
Fiji OK
Gabon OK
Gambia OK
Georgia OK
Ghana OK
Greece OK
Grenada OK
Guatemala OK
Guinea OK
Guinea-Bissau OK
Guyana OK
Haiti OK
Honduras OK
India OK
Indonesia OK
Iraq OK
Jamaica OK
Jordan OK
Kazakhstan OK
Kenya OK
Kiribati OK
Kosovo OK
Kuwait OK
Kyrgyzstan OK
Lao People's Democratic Republic NOT IN LANDS
Lebanon OK
Lesotho OK
Libya OK
Madagascar OK
Malawi OK
Maldives OK
Mali OK
Mar

In [23]:
lands["ADMIN"] = lands["ADMIN"].replace(["Bolivia"], "Bolivia (Plurinational State of)")
lands["ADMIN"] = lands["ADMIN"].replace(["Republic of the Congo"], "Congo")
lands["ADMIN"] = lands["ADMIN"].replace(["eSwatini"], "Eswatini")
lands["ADMIN"] = lands["ADMIN"].replace(["Laos"], "Lao People's Democratic Republic")
lands["ADMIN"] = lands["ADMIN"].replace(["Federated States of Micronesia"], "Micronesia (Federated States of)")
lands["ADMIN"] = lands["ADMIN"].replace(["Macedonia"], "North Macedonia")
lands["ADMIN"] = lands["ADMIN"].replace(["Moldova"], "Republic of Moldova")
lands["ADMIN"] = lands["ADMIN"].replace(["S√£o Tom√© and Principe"], "Sao Tome and Principe")
lands["ADMIN"] = lands["ADMIN"].replace(["Republic of Serbia"], "Serbia")
lands["ADMIN"] = lands["ADMIN"].replace(["Palestine"], "State of Palestine")
lands["ADMIN"] = lands["ADMIN"].replace(["Syria"], "Syrian Arab Republic")
lands["ADMIN"] = lands["ADMIN"].replace(["East Timor"], "Timor-Leste")
lands["ADMIN"] = lands["ADMIN"].replace(["Syria"], "Syrian Arab Republic")



In [24]:
land = list(lands['ADMIN'].unique())

for c in unicef:
    if c in land: print(c, "OK")
    else: print(c, "NOT IN LANDS")

Afghanistan OK
Albania OK
Anguilla OK
Antigua and Barbuda OK
Armenia OK
Bangladesh OK
Barbados OK
Belarus OK
Belize OK
Benin OK
Bhutan OK
Bolivia (Plurinational State of) OK
Bosnia and Herzegovina OK
Botswana OK
Brazil OK
British Virgin Islands OK
Bulgaria OK
Burkina Faso OK
Burundi OK
Cameroon OK
Central African Republic OK
Chad OK
Chile OK
Colombia OK
Comoros OK
Congo OK
Cook Islands OK
Costa Rica OK
Croatia OK
Cuba OK
Democratic Republic of the Congo OK
Djibouti OK
Dominica OK
Dominican Republic OK
Ecuador OK
Egypt OK
El Salvador OK
Equatorial Guinea OK
Eswatini OK
Ethiopia OK
Fiji OK
Gabon OK
Gambia OK
Georgia OK
Ghana OK
Greece OK
Grenada OK
Guatemala OK
Guinea OK
Guinea-Bissau OK
Guyana OK
Haiti OK
Honduras OK
India OK
Indonesia OK
Iraq OK
Jamaica OK
Jordan OK
Kazakhstan OK
Kenya OK
Kiribati OK
Kosovo OK
Kuwait OK
Kyrgyzstan OK
Lao People's Democratic Republic OK
Lebanon OK
Lesotho OK
Libya OK
Madagascar OK
Malawi OK
Maldives OK
Mali OK
Marshall Islands OK
Mauritania OK
Mexico OK